Langkah 1 Import Library

In [82]:
import pandas as pd
from transformers import  AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import Dataset

Langkah 2

In [83]:
df_train = pd.read_csv("ada labelnya - Sheet1.csv")
df_train = df_train.rename(columns={"Data Komentar TIKTOK": "text", "Labeling": "label"})
label_map = {"negatif": 0, "netral": 1, "positif": 2}
df_train = df_train[df_train["label"].isin(label_map.keys())]
df_train["label"] = df_train["label"].map(label_map)

df_train.head()

,text,label
0,Aku sebagai pemakai Hb noera bangga banget con...,2
1,mending noera apa lauskin?,1
2,tapi kok di aku ga ngaruh,0
3,"ikut gemeteran karena aku pake noera, ini cepe...",2
4,Yeayy bangga banget sebagai pemakai hb noera,2


In [84]:
# print("Dataset Unlabeled:")
# df_unlabeled.head()

df_test = pd.read_csv("gaada labelnya - Sheet1.csv")
df_test = df_test.rename(columns={"Data Komentar TIKTOK": "text"})

df_test.head()

,text,Labeling
0,dok moisturizer Marina dong,NaN
1,Muar bngt bila llu d beranda q,NaN
2,angel tacik dok!,NaN
3,Bu doktif review minuman Collagen juga dongg,NaN
4,DOK COBA BRAND CAMILLE PUNYA KAK,NaN


Langkah 3

In [85]:
model_name = "cahya/roberta-base-indonesian-522M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

/Users/habibfauzanmahardika/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /Users/habibfauzanmahardika/.cache/huggingface/hub/models--cahya--roberta-base-indonesian-522M/snapshots/88447f4cf0e27ca82cb25b7d841f9add236b08f3/config.json
Model config RobertaConfig {
  "_name_or_path": "cahya/roberta-base-indonesian-522M",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "n

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

Langkah 4

In [86]:
train_dataset = Dataset.from_pandas(df_train)

def preprocess_function(examples):
    texts = examples["text"]
    texts = [str(t) for t in texts]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/502 [00:00<?, ? examples/s]

Langkah 5

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Langkah 6

In [88]:
trainer.train()

/Users/habibfauzanmahardika/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 502
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 189
  Number of trainable parameters = 125979650


  0%|          | 0/189 [00:00<?, ?it/s]

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.


IndexError: Target 2 is out of bounds.

Langkah 7

In [ ]:
model.eval()

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        pred_label = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred_label].item()
    return pred_label, confidence

sentiments = []
confidences = []

for text in tqdm(df_test["text"]):
    label_id, conf = predict_sentiment(str(text))
    label_str = {0: "negatif", 1: "netral", 2: "positif"}[label_id]
    sentiments.append(label_str)
    confidences.append(conf)

df_test["predicted_sentiment"] = sentiments
df_test["confidence"] = confidences

Langkah 8 Simpan hasil sentimen ke file excel baru

In [ ]:
df_test.to_csv("hasil_prediksi_sentimen.csv", index=False)
print("✅ Selesai! Hasil disimpan ke hasil_prediksi_sentimen.csv")

Hasil disimpan di: hasil_sentimen_unlabeled.csv
